# HDFS Log data Loading 

### https://github.com/logpai/loghub/tree/master/HDFS

In [1]:
# count rows
with open('project_raw/HDFS.log', "r") as file:
    totaln=0
    for line in file:
        totaln += 1

In [2]:
print('There are a total of {} lines'.format(totaln))

There are a total of 11175629 lines


In [3]:
#Take a quick look at the data:
   
data = []
with open('project_raw/HDFS.log', "r") as file:
    n=0
    for line in file:
        data.append(line)
        if n <200:
            n += 1
        else: break 

In [6]:
import pandas as pd

df = pd.DataFrame(data)
df.head()

,0
0,081109 203518 143 INFO dfs.DataNode$DataXceive...
1,081109 203518 35 INFO dfs.FSNamesystem: BLOCK*...
2,081109 203519 143 INFO dfs.DataNode$DataXceive...
3,081109 203519 145 INFO dfs.DataNode$DataXceive...
4,081109 203519 145 INFO dfs.DataNode$PacketResp...


In [7]:
df.iloc[100:110].values

array([['081109 203527 154 INFO dfs.DataNode$DataXceiver: 10.251.197.226:50010 Served block blk_-3544583377289625738 to /10.251.203.4\n'],
       ['081109 203527 154 INFO dfs.DataNode$DataXceiver: 10.251.215.16:50010 Served block blk_-1608999687919862906 to /10.250.19.227\n'],
       ['081109 203527 155 INFO dfs.DataNode$DataXceiver: 10.250.11.100:50010 Served block blk_-3544583377289625738 to /10.250.19.227\n'],
       ['081109 203527 155 INFO dfs.DataNode$DataXceiver: 10.251.197.226:50010 Served block blk_-3544583377289625738 to /10.251.215.16\n'],
       ['081109 203527 156 INFO dfs.DataNode$DataXceiver: 10.250.11.100:50010 Served block blk_-3544583377289625738 to /10.251.65.203\n'],
       ['081109 203527 156 INFO dfs.DataNode$DataXceiver: 10.251.197.226:50010 Served block blk_-3544583377289625738 to /10.250.17.177\n'],
       ['081109 203527 157 INFO dfs.DataNode$DataXceiver: 10.250.11.100:50010 Served block blk_-3544583377289625738 to /10.251.66.63\n'],
       ['081109 203527 157

In [41]:
df.head(10)

,0
0,081109 203518 143 INFO dfs.DataNode$DataXceive...
1,081109 203518 35 INFO dfs.FSNamesystem: BLOCK*...
2,081109 203519 143 INFO dfs.DataNode$DataXceive...
3,081109 203519 145 INFO dfs.DataNode$DataXceive...
4,081109 203519 145 INFO dfs.DataNode$PacketResp...
5,081109 203519 145 INFO dfs.DataNode$PacketResp...
6,081109 203519 145 INFO dfs.DataNode$PacketResp...
7,081109 203519 145 INFO dfs.DataNode$PacketResp...
8,081109 203519 147 INFO dfs.DataNode$PacketResp...
9,081109 203519 147 INFO dfs.DataNode$PacketResp...


Split the data into a training and testing set based on 80/20 split. Using the ordered data (no shuffling or random selection) as they are based on a log series history and we want to use the last 20% for test and we want to maintain the order of data.

In [8]:
train_idx = int(totaln*.8)
train_idx

8940503

In [9]:
# read the training lines only
train_data = []
with open('project_raw/HDFS.log', "r") as file:
    n=0
    for line in file:
        if n < train_idx:
            train_data.append(line)
            n += 1
        else: break 

In [10]:
# write to the training file `HDFS_train.log`
with open('project_raw/HDFS_train.log', 'x') as file:
    for i in train_data:
        file.write(i)

## Log Parsing using Drain (Run log_parser)

In [29]:
all_parsed = pd.read_csv('project_parsed/HDFS.log_structured.csv')

In [31]:
all_parsed[0:]

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,"['blk_-1608999687919862906', '/10.250.19.102:5..."
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,['/mnt/hadoop/mapred/system/job_200811092030_0...
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,"['blk_-1608999687919862906', '/10.250.10.6:405..."
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,"['blk_-1608999687919862906', '/10.250.14.224:4..."
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,d38aa58d,PacketResponder <*> for block <*> <*>,"['1', 'blk_-1608999687919862906 terminating']"
...,...,...,...,...,...,...,...,...,...,...
11175624,11175625,81111,111557,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_-61713680325832...,32777b38,Verification succeeded for <*>,['blk_-6171368032583208892']
11175625,11175626,81111,111607,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_619502527611431...,32777b38,Verification succeeded for <*>,['blk_6195025276114316035']
11175626,11175627,81111,111613,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_-33397734047143...,32777b38,Verification succeeded for <*>,['blk_-3339773404714332088']
11175627,11175628,81111,111615,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_103723194550928...,32777b38,Verification succeeded for <*>,['blk_1037231945509285002']


In [43]:
all_parsed['Component']

0               dfs.DataNode$DataXceiver
1                       dfs.FSNamesystem
2               dfs.DataNode$DataXceiver
3               dfs.DataNode$DataXceiver
4           dfs.DataNode$PacketResponder
                        ...             
11175624            dfs.DataBlockScanner
11175625            dfs.DataBlockScanner
11175626            dfs.DataBlockScanner
11175627            dfs.DataBlockScanner
11175628            dfs.DataBlockScanner
Name: Component, Length: 11175629, dtype: object

In [32]:
a = pd.read_csv("./project_parsed/HDFS.log_templates.csv")

In [33]:
a

,EventId,EventTemplate,Occurrences
0,09a53393,Receiving block <*> src: <*> dest: <*>,1723232
1,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,575061
2,d38aa58d,PacketResponder <*> for block <*> <*>,1706728
3,e3df2680,Received block <*> of size <*> from <*>,1706514
4,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap upd...,1719741
5,d6b7b743,Received block <*> src: <*> dest: <*> of size <*>,7097
6,73c2ec69,<*>Transmitted block <*> to <*>,6937
7,75627efd,<*> Starting thread to transfer block <*> to <...,165
8,54e5f6b4,BLOCK* ask <*> to replicate <*> to datanode(s)...,165
9,626085d5,<*> Served block <*> to <*>,428726


In [27]:
test_parsed = all_parsed.iloc[train_idx:]

In [28]:
test_parsed.to_csv('project_parsed/HDFS_test.log_structured.csv', index = False)

In [34]:
test_parsed

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
8940503,8940504,81111,73331,23775,INFO,dfs.DataNode$DataXceiver,Receiving block blk_4260393861082877150 src: /...,09a53393,Receiving block <*> src: <*> dest: <*>,"['blk_4260393861082877150', '/10.251.199.245:3..."
8940504,8940505,81111,73331,26,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap upd...,"['10.251.27.63:50010', 'blk_109129139965973019..."
8940505,8940506,81111,73331,26,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap upd...,"['10.251.30.85:50010', 'blk_460404685393070049..."
8940506,8940507,81111,73331,27,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap upd...,"['10.251.126.227:50010', 'blk_1091291399659730..."
8940507,8940508,81111,73331,27,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap upd...,"['10.251.27.63:50010', 'blk_-35992286980708344..."
...,...,...,...,...,...,...,...,...,...,...
11175624,11175625,81111,111557,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_-61713680325832...,32777b38,Verification succeeded for <*>,['blk_-6171368032583208892']
11175625,11175626,81111,111607,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_619502527611431...,32777b38,Verification succeeded for <*>,['blk_6195025276114316035']
11175626,11175627,81111,111613,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_-33397734047143...,32777b38,Verification succeeded for <*>,['blk_-3339773404714332088']
11175627,11175628,81111,111615,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_103723194550928...,32777b38,Verification succeeded for <*>,['blk_1037231945509285002']


In [35]:
test_parsed.shape

(2235126, 10)

In [37]:
train_data = pd.read_csv("./project_parsed/HDFS_train.log_structured.csv")

In [39]:
train_data.shape

(8940503, 10)